In [1]:
import pyodbc
import pandas as pd
print(pyodbc.drivers()) 

['SQL Server', 'SQL Server Native Client 11.0', 'ODBC Driver 17 for SQL Server']


In [ ]:
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=LEINADSAMA\\LEINADSERVER;'
    'DATABASE=LeiMedicalDB;'
    'Trusted_Connection=yes;'
)

In [ ]:
query = """
    SELECT 
        p.patient_id, p.name, p.gender, p.blood_type,
        a.admission_date, a.discharge_date, a.diagnosis,
        d.name AS doctor_name, d.specialization,
        COUNT(l.test_id) AS num_lab_tests
    FROM 
        Patients p
    JOIN 
        Admissions a ON p.patient_id = a.patient_id
    JOIN 
        Doctors d ON a.doctor_id = d.doctor_id
    LEFT JOIN 
        Lab_Results l ON p.patient_id = l.patient_id
    GROUP BY 
        p.patient_id, p.name, p.gender, p.blood_type,
        a.admission_date, a.discharge_date, a.diagnosis,
        d.name, d.specialization;
"""


In [ ]:
df = pd.read_sql(query, conn)
df.to_csv('patient_admissions_details.csv', index=False)
# conn.close()

In [ ]:
query = """
    SELECT
        a.diagnosis,
        p.gender,
        CASE 
            WHEN DATEDIFF(YEAR, p.dob, GETDATE()) BETWEEN 0 AND 18 THEN '0-18'
            WHEN DATEDIFF(YEAR, p.dob, GETDATE()) BETWEEN 19 AND 35 THEN '19-35'
            WHEN DATEDIFF(YEAR, p.dob, GETDATE()) BETWEEN 36 AND 50 THEN '36-50'
            WHEN DATEDIFF(YEAR, p.dob, GETDATE()) BETWEEN 51 AND 65 THEN '51-65'
            ELSE '65+'
        END AS age_group,
        COUNT(*) AS prevalence_count
    FROM 
        Admissions a
    JOIN 
        Patients p ON a.patient_id = p.patient_id
    GROUP BY 
        a.diagnosis, p.gender,
        CASE 
            WHEN DATEDIFF(YEAR, p.dob, GETDATE()) BETWEEN 0 AND 18 THEN '0-18'
            WHEN DATEDIFF(YEAR, p.dob, GETDATE()) BETWEEN 19 AND 35 THEN '19-35'
            WHEN DATEDIFF(YEAR, p.dob, GETDATE()) BETWEEN 36 AND 50 THEN '36-50'
            WHEN DATEDIFF(YEAR, p.dob, GETDATE()) BETWEEN 51 AND 65 THEN '51-65'
            ELSE '65+'
        END
    ORDER BY 
        a.diagnosis, age_group, p.gender;
"""

In [ ]:
df = pd.read_sql(query, conn)
df.to_csv('disease_prevalence_by_age_gender.csv', index=False)
conn.close()
